### Basic Filtering (retain relevant HScode rows and name columns)

#### The code below first filters the rows to only retain the rows with valid HS codes : 03, 3, 1604, 1605. Then, any rows with HS Code starting with '3' will be edited to start with '03'. 

In [39]:
import pandas as pd
import os
import csv
import sys

# Increase the maximum field size limit
csv.field_size_limit(sys.maxsize)

# 1. File paths for your model outputs
output_filepaths = {
    "panjiva_us_imports_01_2015": "/home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_01_2015.csv",
    "panjiva_us_imports_02_2015": "/home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_02_2015.csv",
    "panjiva_us_imports_03_2015": "/home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_03_2015.csv"
}

# Dictionary to store filtered DataFrames
filtered_dataframes = {}

# 2. Loop through files and process
for model, filepath in output_filepaths.items():
    print(f"Processing {model} from {filepath}...")
    try:
        df = pd.read_csv(filepath, engine="python")
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        continue

    print(f"{model}: Loaded {len(df)} rows")

    # --- Save original column names ---
    original_columns = df.columns.copy()

    # --- Normalize column names ---
    df.columns = df.columns.str.strip().str.replace('\ufeff', '', regex=True)  # Remove BOM
    df.columns = df.columns.str.lower()  # Lowercase for easy match

    # Check if HSCode column exists after normalization
    if 'hscode' not in df.columns:
        print(f"Skipping {model} - 'HSCode' column not found after normalization.")
        continue

    # --- Clean HSCode column ---
    df['hscode'] = df['hscode'].astype(str).str.strip()
    df['hscode'] = df['hscode'].str.replace('.', '', regex=False)  # Remove periods

    # Add leading zero if starts with '3' but not already '03'
    df['hscode'] = df['hscode'].apply(lambda x: '0' + x if x.startswith('3') and not x.startswith('03') else x)

    # --- Filtering function ---
    def hscode_matches(cell):
        codes = [code.strip() for code in cell.split(';')]
        for code in codes:
            if code.startswith('03') or code.startswith('1604') or code.startswith('1605'):
                return True
        return False

    # Filter rows based on HSCode match
    filtered_df = df[df['hscode'].apply(hscode_matches)]

    # --- Restore original column names ---
    filtered_df.columns = original_columns

    # --- Save filtered file ---
    base_dir = os.path.dirname(filepath)
    out_path = os.path.join(base_dir, f"{model}_filtered_HSCode.csv")
    filtered_df.to_csv(out_path, index=False)

    # Store and display filtered DataFrame
    filtered_dataframes[model] = filtered_df
    print(f"Saved to {out_path}")
    print(f"{model}: {filtered_df.shape[0]} rows")
    display(filtered_df)

print("\n--- HSCode filtering complete ---")


Processing panjiva_us_imports_01_2015 from /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_01_2015.csv...
panjiva_us_imports_01_2015: Loaded 68536 rows
Skipping panjiva_us_imports_01_2015 - 'HSCode' column not found after normalization.
Processing panjiva_us_imports_02_2015 from /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_02_2015.csv...
panjiva_us_imports_02_2015: Loaded 802028 rows
Saved to /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_02_2015_filtered_HSCode.csv
panjiva_us_imports_02_2015: 95904 rows


,PanjivaRecordID,BillOfLadingNumber,ArrivalDate,DataLoadDate,DataLaunchDate,ConsigneeName,ConsigneeFullAddress,ConsigneeRoute,ConsigneeCity,ConsigneeStateRegion,...,HasLCL,ContainerNumbers,HSCode,GoodsShipped,VolumeContainerTEU,ContainerMarks,DividedLCL,ContainerTypeOfService,ContainerTypes,DangerousGoods
21,107625726,MOLU11032251740,2015-02-03,2015-02-04,2015-02-06,Haworth Inc.,316 HIGHWAY 9 S BRUCE MISSISSIPPI 389159781,Highway 9,Bruce,Mississippi,...,NaN,MOEU1409807,0391733,"168 CTN CHAIR PARTS ARM ASSY,HEIGHT ADJ,TR_F ...",0.4,"168 CTN CHAIR PARTS ARM ASSY,HEIGHT ADJ,TR_F ...",Y,House to House,9500,false
26,107753726,ASTI75524127999,2015-02-08,2015-02-09,2015-02-11,Republic Tobacco,2301 RAVINE WAY GLENVIEW IL 60025 USA,2301 Ravine Way,Glenview,Illinois,...,NaN,TGHU4143476,0320300,COLOR BOX . .,2.0,"R T L.P 2301 RAVINE WAY GLENVIEW ,IL 60025 IT ...",N,Pier to Pier,NaN,false
37,107686654,COSU6107751810,2015-02-04,2015-02-05,2015-02-07,Fulton Seafood Inc.,2818 MCKINNEY ST HOUSTON TEXAS 77003,2818 McKinney Street,Houston,Texas,...,NaN,CBHU2817521,030323,FROZEN GUTTED AND SCALED TILAPIA,2.0,NaN,N,House to House,4532,false
41,107662654,YMLUW234009404,2015-02-01,2015-02-03,2015-02-05,Elegant Windows,"3709 EAST RANDOL MILL ROAD ARLINGTON, TX 76011",3709 E Randol Mill Rd,Arlington,Texas,...,NaN,YMLU3264973,0392112,PLATES SHEETS ETC STRIP CELL PLYM VINY ...,1.0,"A.F.P. (IN DIA.) WHITTIER,CA PO028081-00 CTN1-...",N,House to House,22G0,false
45,107644350,OWLQHK1400485,2015-02-03,2015-02-04,2015-02-06,Quirch Foods,7600 NW 82ND PLACE MIAMI,NaN,NaN,NaN,...,NaN,SEGU9052257,030461,FROZEN TILAPIA FILLETS PO NO. 394814 HS CODE IUS,2.0,030461 CONTAINER IS SET AT -18 DEGREES CELS,N,Pier to Pier,NaN,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801996,108622946,CPLBKAEAKFG01,2015-02-28,2015-03-03,2015-03-05,Maritime Products,11825 ROCK LANDING ROAD SUITE Y-1A NEWPORT NEW...,NaN,NaN,NaN,...,NaN,CSVU7506605,030617,"2184 CARTONS: 14.125,10 NET KG. FROZEN IQF SHR...",2.0,BRAND OCEAN BONNIE,N,House to House,4530,false
801999,108664162,KSCTKOSR215FUS14,2015-02-28,2015-03-03,2015-03-05,Englewood Lab,88 WEST SHEFFIELD AV ENGLEWOOD NJ 07631 USA,88 West Sheffield Avenue,Englewood,New Jersey,...,True,TEMU9077816,0330420,2PKG OF COSMETIC CONTAINERS,0.06,NaN,Y,Container Yard,4CR0,false
802000,108667554,MAEU603786084,2015-02-28,2015-03-03,2015-03-05,Ch Robinson,855 ARTHUR AV ELK GROVE VILLAGE IL 60007,NaN,NaN,NaN,...,NaN,MSKU0295735,0392410,BABY CARE PRODUCTS 9OZ PLYTM INS SPOUT CUP ...,2.0,PLAYTEX MADE IN CHINA PO# ITEM# DATE CODE: ...,N,Container Yard,4EB0,false
802022,108701666,SNYLULSSHZ48097,2015-02-28,2015-03-03,2015-03-05,Amz Express Inc.,3000 VILLAGE RUN RD STE 103 #178 WEXFORD PA 15090,3000 Village Run Road,Wexford,Pennsylvania,...,True,PONU7579081,0392690,"11INCH NEOPRENE CASE, 13INCH NEOPRENE CASE, 10...",0.42,1/472...472/472 AMZEX MADE IN CHINA,Y,Container Yard,4EG0,false


Processing panjiva_us_imports_03_2015 from /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_03_2015.csv...
panjiva_us_imports_03_2015: Loaded 991362 rows
Saved to /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_03_2015_filtered_HSCode.csv
panjiva_us_imports_03_2015: 116665 rows


,PanjivaRecordID,BillOfLadingNumber,ArrivalDate,DataLoadDate,DataLaunchDate,ConsigneeName,ConsigneeFullAddress,ConsigneeRoute,ConsigneeCity,ConsigneeStateRegion,...,HasLCL,ContainerNumbers,HSCode,GoodsShipped,VolumeContainerTEU,ContainerMarks,DividedLCL,ContainerTypeOfService,ContainerTypes,DangerousGoods
20,108982847,HYSLNNGB15020886,2015-03-07,2015-03-10,2015-03-12,Nevada Crystal Premium,6185 S VALLEY VIEW BLVD SUITEH LAS VEGAS NV 89118,6185 South Valley View Boulevard,Las Vegas,Nevada,...,NaN,MSCU9702827,0392330,PET PREFORM 5 GALLON CAP 28MM CAP GLASS BOTTLE,2.0,NaN,N,House to House,4510,false
21,109064575,SHPTKYCHI14630,2015-03-06,2015-03-10,2015-03-12,Taap Corp.,300 HOLBROOK DR WHEELING IL 60090 USA,300 Holbrook Drive,Wheeling,Illinois,...,True,TCLU9826028,0392690,DC MICRO MOTOR,0.11,PO 6334 035 10,Y,NaN,45G0,false
22,109005375,MAEU952875299,2015-03-09,2015-03-10,2015-03-12,Channel Seafoods International,4755 TECHNOLOGY WAY BOCA RATON FL 33431,4755 Technology Way,Boca Raton,Florida,...,NaN,MNBU3414224,160510,1700CTNS OF FROZEN SEAFOOD MIX CONTRACT NO. ...,2.0,NaN,N,Container Yard,4ER0,false
23,108694911,PNEP15598452239,2015-03-01,2015-03-03,2015-03-05,Sigma Aldrich International Gmbh,"3050 SPRUCE STREET ST. LOUIS, MO 63103-2530",NaN,NaN,NaN,...,True,TCLU4236420,0390599,PVP360-BULK POLYVINYLPYRROLIDONE,0.02,SIGMA-ALDRICH INTERNATIONAL GMBH,Y,House to House,42G1,false
28,108986431,JPSAWLUS11261,2015-03-08,2015-03-10,2015-03-12,Fireworks Forever,2174 HWY 8 SAINT CROIX FALLS SAINT CROIX FALLS...,2174 U.S. 8,Saint Croix Falls,Wisconsin,...,NaN,MATU2579767,0360410,"UN0336,FIREWORKS,1.4G,PGII",2.0,NaN,N,Pier to Pier,4400,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991318,109923661,ZIMUBCN0055365,2015-03-29,2015-03-30,2015-04-01,Fluidra Usa Llc,8525 MALLORY RD JACKSONVILLE FL32 220 USA,NaN,NaN,NaN,...,NaN,ZCSU8769619,0392690,SWIMMING POOL EQUIPMENTS CEPEX - 765 BUL KS / ...,2.0,NO NUMBERS,N,House to House,4500,false
991335,109837965,HJSCSEL519600A01,2015-03-27,2015-03-31,2015-04-02,Lg Chem America Inc.,910 SYLVAN AV. ENGLEWOOD CLIFFS NJ 07632 U.S.A,NaN,NaN,NaN,...,NaN,HJCU1498164,0390120,RESIN(SBS),2.0,NaN,N,Pier to House,4FG0,false
991340,109989133,HLCUME3150332869,2015-03-31,2015-04-02,2015-04-04,Pilkington Automotive Argentina,S.A. AV. BERNARDO ADER 3060 1605 MUNRO PROV. D...,NaN,NaN,NaN,...,NaN,HLXU8026702; TRLU8065345,0392091; 392091,POLIVINIL BUTIRAL FILM 25 PALLETS WITH 100 ROL...,2.0; 2.0,. . . . . .; . . . . . .,N; N,House to House; House to House,4510; 4510,false; false
991349,110010701,OERT202701E00985,2015-03-29,2015-04-03,2015-04-05,Prodlink,333 NORTH BALDWIN PARK BOULEVARD CITY OF INDUS...,333 North Baldwin Park Boulevard,City of Industry,California,...,NaN,CMAU5843487,0392310,"PLASTIC CD & DVD CASES / CLEAR TAPE PO#:5982 ""...",2.0,NaN,N,Pier to Pier,4500,false



--- HSCode filtering complete ---


####

#### The code below retains only relevant name columns.
#### US imports : ConsigneeName, ConsigneeLocalDUNS, ConsigneePanjivaID, ShipperName, ShipperPanjivaID

In [40]:
# Columns to keep
columns_to_keep = [
    "HSCode",
    "ConsigneeName",
    "ConsigneeLocalDUNS",
    "ConsigneePanjivaID",
    "ShipperName",
    "ShipperPanjivaID",
]

# Loop over filtered DataFrames from Section 1
for model, df_filtered in filtered_dataframes.items():
    final_df = df_filtered[columns_to_keep]
    
    out_path = os.path.join("/home/nikhikhas/TradeSweep/datasets", f"{model}_filtered_HSCode_selected_columns.csv")
    final_df.to_csv(out_path, index=False)
    
    print(f"Saved {model} selected columns to {out_path}")
    display(final_df)


Saved panjiva_us_imports_02_2015 selected columns to /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_02_2015_filtered_HSCode_selected_columns.csv


,HSCode,ConsigneeName,ConsigneeLocalDUNS,ConsigneePanjivaID,ShipperName,ShipperPanjivaID
21,0391733,Haworth Inc.,NaN,27837534.0,Changzhou Viscount Plastic,27799361.0
26,0320300,Republic Tobacco,65889925.0,1846393.0,Ocean Packaging Paper Products,45730327.0
37,030323,Fulton Seafood Inc.,137544938.0,1982366.0,Xiamen Huison Foods Co.,45342071.0
41,0392112,Elegant Windows,969938810.0,1966551.0,Beautiful Window Enterprise,45988165.0
45,030461,Quirch Foods,45467826.0,27824514.0,Guangxi Nanning Baiyang Food Co.,5096630.0
...,...,...,...,...,...,...
801996,030617,Maritime Products,133554415.0,27808492.0,Pesquera Veraz S.A.,1215928.0
801999,0330420,Englewood Lab,78461960.0,27885419.0,Fine.Com,28373276.0
802000,0392410,Ch Robinson,783698863.0,27804863.0,NaN,NaN
802022,0392690,Amz Express Inc.,NaN,27970628.0,Wish Trading Company,44444311.0


Saved panjiva_us_imports_03_2015 selected columns to /home/nikhikhas/TradeSweep/datasets/panjiva_us_imports_03_2015_filtered_HSCode_selected_columns.csv


,HSCode,ConsigneeName,ConsigneeLocalDUNS,ConsigneePanjivaID,ShipperName,ShipperPanjivaID
20,0392330,Nevada Crystal Premium,132633616.0,2507816.0,Es Aqua Limited,28149409.0
21,0392690,Taap Corp.,615113065.0,27836127.0,Zhejiang Zhenqi Auto Parts Corp. Lt,5463518.0
22,160510,Channel Seafoods International,131512154.0,44238862.0,"Shantou Ruiyuan Industries Co., Ltd.",2166589.0
23,0390599,Sigma Aldrich International Gmbh,NaN,27830917.0,Sigma Aldrich Wuxi Life Science,25464957.0
28,0360410,Fireworks Forever,884643735.0,1988445.0,Hunan Top Ideal Supply Chain Servic,29793304.0
...,...,...,...,...,...,...
991318,0392690,Fluidra Usa Llc,NaN,40864660.0,Trace Logistics S.A.,42239803.0
991335,0390120,Lg Chem America Inc.,194655882.0,27821413.0,Lg Chem Ltd.,44211817.0
991340,0392091; 392091,Pilkington Automotive Argentina,NaN,40769187.0,Sekisui S Lec Mexico,44209195.0
991349,0392310,Prodlink,808331693.0,1841199.0,"Wizpak P & P Co., Ltd.",28325740.0


####

#### The final output is a csv file with only rows with valid HS Codes retained as well as only relevant name columns retained.